# Chat Driver

An OpenAI Chat Completions API wrapper.

## Notebook setup

Run this cell to set the notebook up. Other sections can be run independently.

In [1]:
%reload_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from azure.identity import aio, DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential

from openai import AsyncAzureOpenAI, AzureOpenAI

import logging 
import json
from pathlib import Path

# Set up structured logging to a file.
class JsonFormatter(logging.Formatter):
    def format(self, record) -> str:
        record_dict = record.__dict__
        log_record = {
            'timestamp': self.formatTime(record, self.datefmt),
            'level': record.levelname,
            'session_id': record_dict.get('session_id', None),
            'run_id': record_dict.get('run_id', None),
            'message': record.getMessage(),
            'data': record_dict.get('data', None),
            'module': record.module,
            'funcName': record.funcName,
            'lineNumber': record.lineno,
            'logger': record.name,
        }
        extra_fields = {
            key: value for key, value in record.__dict__.items() 
            if key not in ['levelname', 'msg', 'args', 'exc_info', 'funcName', 'module', 'lineno', 'name', 'message', 'asctime', 'session_id', 'run_id', 'data']
        }
        log_record.update(extra_fields)
        return json.dumps(log_record)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)
if logger.hasHandlers():
    logger.handlers.clear()
data_dir = Path('.data')
if not data_dir.exists():
    data_dir.mkdir()
handler = logging.FileHandler(data_dir / 'logs.jsonl')
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)


load_dotenv()
credential = DefaultAzureCredential()

azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT", ""),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION", ""),
    "max_retries": 2,
}

model = azure_openai_config.get("azure_deployment", "gpt-4o")

async_client = AsyncAzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=aio.get_bearer_token_provider(
        aio.AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

client = AzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=get_bearer_token_provider(
        AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

## ChatCompletionsAPI usage

Azure/OpenAI's Chat Completions API is the fundamental building block of an AI assistant that uses the GPT model. 

- https://platform.openai.com/docs/api-reference/chat
- https://github.com/openai/openai-python/blob/main/api.md
- https://platform.openai.com/docs/api-reference/chat drivers

### Sync

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
)
print(completion.model_dump())

### Async

In [ ]:
response = await async_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
)
print(response)

### Streaming

In [ ]:
stream = await async_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
    stream=True,
)
async for chunk in stream:
    print(chunk.model_dump())

## OpenAI Chat Completion Driver (a.k.a "chat driver")

### OpenAI Assistants

The Azure/OpenAI Assistants API is newer, stateful API that splits an `assistant` from the data about a conversation `thread` that can be `run` against an `assistant`. Additionally, you can add `tools` to an assistant that enable the assistant to have more interactive capabilities. The tools currently available are:

- *Functions*: Registering local functions with the assistant so it knows it can call them before generating a response. This is a "hold on let me look that up for you" kind of interaction.
- *File Search* (formerly the retrieval plugin): Attach one or more files and they will be RAG-vectorized and available as content to the assistant.
- *Code Interpreter*: Run python code in a secure sandbox.

The Assistant API productized as OpenAI's `GPTs` product. The `GPT Builder` lets developers create and deploy GPTs assistants using a web interface.

### Chat Driver

But an "assistant" requires pretty strong "abstraction lock-in". This thing isn't really an assistant in the fullest sense... it's more like a "pseudo-assistant", but this confuses things. Let's just let the Chat Completion API be what it is and drive it as necessary as we create our assistants. Let's just wrap up the function calling bits (which, ultimately, can give you the other tools like Functions and File Search) in a simple-to-use GPT-like interface we'll call a *chat driver*.

The chat driver is meant to be used the exact way the Chat Completions API is... just easier.

Our chat driver provides:

- The ability to almost magically register functions to the function tool using a `FunctionRegistry`.
- Tracking of message history.
- Management of a `Context` object that can be used for session management and supply additional context to functions.
- Some prompt creation helpers.
- Other utilities... this is just meant to be an interface you can use to forget about all the api complexities.

In [2]:
from typing import cast
from chat_driver import ChatDriver, ChatDriverConfig, Context
from chat_driver import LocalMessageHistoryProvider #, LocalMessageHistoryProviderConfig
# from typing import List
# from openai.types.chat import ChatCompletionMessageParam


# When an chat driver is created, it will automatically create a context with a
# session_id. Or, if you want to use a specific session_id, you can pass it as
# an argument. This is useful for scoping this chat driver instance to an
# external identifier.
context = Context("conversation-id-1000")


# Define tool functions for the chat driver. All functions used by the chat driver
# require a session_id as the first argument.
def get_file_contents(context: Context, file_path: str) -> str:
    """Return the contents of a file."""
    return "The purpose of life is to be happy."


def erase(context: Context, name: str) -> str:
    """Erases a stored value."""
    return f"{context.session_id}: {name} erased"


# Define the chat driver.
instructions = "You are a helpful assistant."

# Define the conversation so far (optional).
# messages: List[ChatCompletionMessageParam] = []
# localMessageHistoryConfig = LocalMessageHistoryProviderConfig(f"./data/{context.session_id}", messages)
# message_provider = LocalMessageHistoryProvider(localMessageHistoryConfig)

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
        context=context,
        # message_provider=message_provider,
        commands=[erase],  # Commands can be registered when instantiating the chat driver.
        functions=[erase],  # Functions can be registered when instantiating the chat driver.
    ),
)

# Let's clear the data from previous runs.
message_provider = cast(LocalMessageHistoryProvider, chat_driver.message_provider)
message_provider.delete_all()


# You can also use the `register_function` decorator to register a function.
# Remember, all functions used by the chat driver require a session_id as the
# first argument.
@chat_driver.register_function_and_command
def echo(context: Context, text: str) -> str:
    """Return the text."""
    return f"{context.session_id}: {text}"


# You can also register functions manually.
chat_driver.register_function_and_command(get_file_contents)

# Ok. Let's see if we got one.
print(chat_driver.context.session_id)

# Let's see if the agent can respond.
response = await chat_driver.respond("Hi, my name is Paul.")
print()
print(response.message)

# Help command (shows command available).
response = await chat_driver.respond("/help")
print()
print(response.message)

# We can run any function or command directly.
response = await chat_driver.functions.echo("Hi, my name is Paul.")
print()
print(response)

# Let's see if the chat driver has the ability to run it's own registered function.
response = await chat_driver.respond("Please tell me what's in file 123.txt.")
print()
print(response.message)

# Let's see the full response event.
print()
print(response.to_json())

conversation-id-1000

Hello, Paul! How can I assist you today?

Commands:
help(): Return this help message.
erase(name: str): Erases a stored value.
echo(text: str): Return the text.
get_file_contents(file_path: str): Return the contents of a file.

conversation-id-1000: Hi, my name is Paul.

The contents of the file 123.txt are: "The purpose of life is to be happy." If you need any further assistance or information, feel free to ask!

{
  "id": "0222a092-6bfd-4523-9cf8-d895070c554b",
  "session_id": null,
  "timestamp": "2024-10-10T17:20:00.290438",
  "message": "The contents of the file 123.txt are: \"The purpose of life is to be happy.\" If you need any further assistance or information, feel free to ask!",
  "metadata": {
    "completion_args": {
      "messages": [
        {
          "role": "system",
          "content": "You are a helpful assistant."
        },
        {
          "role": "user",
          "content": "Hi, my name is Paul."
        },
        {
          "conten

## Chat with a chat driver

In [2]:
from typing import Any
from chat_driver import ChatDriverConfig, ChatDriver
from context import Context

context = Context("conversation-id-1001")


def get_file_contents(context: Context, file_path: str) -> str:
    """Returns the contents of a file."""
    return "The purpose of life is to be happy."


def erase(context: Context, name: str) -> str:
    """Erases a stored value."""
    return f"{context.session_id}: {name} erased"


def echo(context: Context, value: Any) -> str:
    """Echos a value as a string."""
    match value:
        case str():
            return value
        case list():
            return ", ".join(map(str, value))
        case dict():
            return json.dumps(value)
        case int() | bool() | float():
            return str(value)
        case _:
            return str(value)


functions = [get_file_contents, erase, echo]

# Define the chat driver.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    context=context,
    commands=functions,
    functions=functions,
)

chat_driver = ChatDriver(chat_driver_config)

# Note: Look in the .data directory for the logs, message history, and other data.

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    response = await chat_driver.respond(message)
    # You can print the entire response event! 
    # print(response.to_json())
    print(f"Assistant: {response.message}", flush=True)

## Chat Driver with an Assistant Drive

In [ ]:
from io import BytesIO
from typing import Any, BinaryIO
from chat_driver import ChatDriverConfig, ChatDriver, ChatDriverConfig
from context import Context
from assistant_drive import Drive, DriveConfig, IfDriveFileExistsBehavior 

session_id = "conversation-id-1001"

context = Context(session_id)

def get_drive_from_context(context):
    return Drive(DriveConfig(root=f".data/drive/{context.session_id}"))

def write_file_contents(context: Context, file_path: str, contents: str) -> str:
    """Writes the contents to a file."""
    drive = get_drive_from_context(context)
    content_bytes: BinaryIO = BytesIO(contents.encode("utf-8"))
    drive.write(content_bytes, file_path, if_exists=IfDriveFileExistsBehavior.OVERWRITE)
    return f"{file_path} updated."

def read_file_contents(context: Context, file_path: str) -> str:
    """Returns the contents of a file."""
    drive = get_drive_from_context(context)
    with drive.open_file(file_path) as file:
        return file.read().decode("utf-8")

functions = [write_file_contents, read_file_contents]

# Define the chat driver.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    context=context,
    commands=functions,
    functions=functions,
)

chat_driver = ChatDriver(chat_driver_config)

# Note: Look in the .data directory for the logs, message history, and other data.

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    response = await chat_driver.respond(message)
    # You can print the entire response event! 
    # print(response.to_json())
    print(f"Assistant: {response.message}", flush=True)